# Лабораторная работа №2. Факторизация больших чисел
Выполнил Ширяев Никита Алексеевич М8О-308Б-22

## Задание
Строку в которой записано своё ФИО подать на вход в качестве аргумента хеш-функции ГОСТ Р 34 11-2012 (Стрибог). Младшие 8 бит выхода интерпретировать как число, которое в дальнейшем будет номером варианта от 0 до 255. В отчёт включить снимок экрана с выбором номера варианта, а также описать шаги решения задачи.
Задача: разложить каждое из чисел $a$ и $b$ на нетривиальные сомножители.

## Вычисление номера варианта
Для вычисления номера варианта, перейдем на [онлайн-шифровщик](https://hashing.tools/streebog/streebog-256) Стрибог и зашифруем свои ФИО.
Возьмем младшие 8 бит (младший байт):

In [1]:
from gostcrypto import gosthash


def get_var_num(fio: str) -> int:
    data = fio.encode("utf-8")
    
    hash_obj = gosthash.new("streebog256", data=data)
    hash_bytes = hash_obj.digest()
    
    last_byte = hash_bytes[-1]
    
    var_num = last_byte
    return var_num


get_var_num("Ширяев Никита Алексеевич")

247

Таким образом, мой номер варианта - 247. В нем следующие числа:

In [ ]:
a = 112326750291640594315099593766598934727832168590250112400930364811750290362153
b = int(
    "323170060713110073007148766886699519604441026697154840321303454275246551388678\
9089319720141152291346368871796092189801949411955915049092109508815245977787610014200\
1251570565968783043737267089990504441778683182473044782658307953092391194442967846344\
8532415839286867571293329740772552939527808050340706617882237170306385037393085505547\
8346036121293991373735247854523758276978185285540912368846792257766716677695284356732\
4922591316237257282271916103116437437085710983522042391465381576095764607003236855472\
9499607761463796173654692210367946897164834989874356048013177153991631165512906281271\
34705141936818293064740240201"
)

Для первого числа можно просто воспользоваться такой Python-библиотекой `sympy`. Она позволяет быстро факторизовать относительно небольшое число.

In [ ]:
import gmpy2
from sympy import factorint


p, q = map(int, factorint(gmpy2.mpz(a)))
print(p)
print(q)

Таким образом, ответ для числа $a$:  
$p=17894683729340745557$  
$q=6277101735386680763835789423207666416102355444464034513029$

Для второго числа программа, использующая такую библиотеку, будет работать слишком долго. Поэтому попробуем найти уязвимости этого числа и проведем несколько популярных атак. Как известно, оба числа имеют только два нетривиальных делителя, как следствие, простых, причем при формировании RSA-ключей, используются простые числа, находящиеся относительно близко друг к другу. Используем эти данные при проведении атак.

## Атака Ферма
Атака Ферма предполагает, что простые множители $p$ и $q$ исходного числа находятся близко друг к другу, то есть $|p - q| < 10^5$. Учитывая тот фактор, что при формировании RSA-ключа могут использоваться близкие друг к другу простые числа, проведем эту атаку:

In [ ]:
import gmpy2


def fermat_factorization(n):
    x = gmpy2.isqrt(n) + 1
    while True:
        y_squared = x**2 - n
        y = gmpy2.isqrt(y_squared)
        if y**2 == y_squared:
            p = x + y
            q = x - y
            return p, q
        x += 1


n = gmpy2.mpz(b)
# fermat_factorization(n)

Этот метод работал слишком долго.

## Атака по общим множителям
Атака по общим множителям подразумевает, что имеется несколько чисел для факторизации и следующее число может иметь один из делителей предыдущего числа.
Скорее всего, было задумано, что факторизовать второе число при текущих ресурсах не получится. Поэтому решение нужно искать на поверхности. Числа в лабораторной работе генерировались не случайно, и, вероятно, среди самих же чисел и есть ответ. Переберем все числа из лабораторной работы и найдем наибольший общий делитель (НОД) с нашим большим числом:

In [ ]:
import math
import re


with open("nums.txt", "r") as file_:
    lines = re.split(r"[ab]\[\d+\]=", file_.read().replace("\n", ""))
    lines.pop(0)

for number in map(int, lines):
    if (div := math.gcd(b, number)) != 1 and b != number:
        print(div)
        print(b // div)
        break

Таким образом, для числа $b$ ответ будет следующим:  
$p=$  
$134078079299425970995740249982058461274793658205923933777235614437217640300735$  
$46976801874298166903427690031858186486050853753882811946569946433649036507369$  
$q=$  
$241031242692103258858011660602831411291209324794568895135967503906525739159180$  
$320066908502410734604966344876628088800478786241697879495832496961298789077465$  
$145521333938162522477078207791768149967684554313738782005759734585790459910946$  
$138712209950796499781564134230067762947335528161742841179416396778587037036896$  
$910922159194305423201156275845008057958785090099371489228347664663118151506380$  
$4873375182260506246992837898705971012525843324401232986857004760600818529$

## Ответ
Для числа $a$:  
$p=17894683729340745557$  
$q=6277101735386680763835789423207666416102355444464034513029$

Для числа $b$:  
$p=$  
$134078079299425970995740249982058461274793658205923933777235614437217640300735$  
$46976801874298166903427690031858186486050853753882811946569946433649036507369$  
$q=$  
$241031242692103258858011660602831411291209324794568895135967503906525739159180$  
$320066908502410734604966344876628088800478786241697879495832496961298789077465$  
$145521333938162522477078207791768149967684554313738782005759734585790459910946$  
$138712209950796499781564134230067762947335528161742841179416396778587037036896$  
$910922159194305423201156275845008057958785090099371489228347664663118151506380$  
$4873375182260506246992837898705971012525843324401232986857004760600818529$

## Выводы
Были получены навыки факторизации больших чисел, которые являются неотъемлемой частью криптографии. Также стало понятным на практике, что решение задачи не всегда имеет математический подход, иногда требуется проанализировать данные, на первый взгляд не относящиеся непосредственно к задаче.